In [1]:
import pandas as pd
import datetime
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff


In [2]:
yandex_russia = pd.read_csv('yandex_russia.csv')

yandex_russia.rename(columns = {'Дата':'Date','Регион':'Region','Заражений':'Confirmed','Выздоровлений':'Recovered',
                               'Смертей':'Deaths','Смертей за день':'Day_deaths',
                               'Заражений за день':'Day_confirmed','Выздоровлений за день':'Day_recovered'},inplace = True)


for i in range(yandex_russia.shape[0]):
    a= yandex_russia.Date[i].split('.')
    yandex_russia.Date[i]=datetime.date(int(a[2]),int(a[1]),int(a[0]))
    
   
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
30913,2020-05-20,Московская обл.,30091,3897,285,14,903,283
30914,2020-09-09,Крым,2891,1947,35,1,46,75
30915,2020-04-24,Камчатский край,84,5,0,0,6,0
30916,2021-02-18,Карачаево-Черкессия,18617,16914,60,0,49,38
30917,2020-12-07,Еврейская АО,3162,2143,51,0,29,15
30918,2020-10-13,Брянская обл.,10446,8264,41,0,86,24
30919,2021-01-11,Смоленская обл.,18783,14287,356,0,155,12
30920,2020-03-31,Архангельская обл.,7,0,0,0,0,0
30921,2021-02-09,Алтайский край,40611,36341,1296,13,174,170
30922,2020-08-31,Томская обл.,5706,4696,69,2,110,15


In [3]:
df3_temp = yandex_russia.groupby('Date').sum()
df3_temp['Region']='Россия'

y5= pd.concat([yandex_russia,df3_temp.reset_index()],axis = 0)
y5.reset_index()
yandex_russia = y5
yandex_russia.tail(10)

,Date,Region,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered
354,2021-03-01,Россия,4257650,3823074,86455,333,11571,11277
355,2021-03-02,Россия,4268215,3838040,86896,441,10565,14966
356,2021-03-03,Россия,4278750,3853734,87348,452,10535,15694
357,2021-03-04,Россия,4290135,3869857,87823,475,11385,16123
358,2021-03-05,Россия,4301159,3885321,88285,462,11024,15464
359,2021-03-06,Россия,4312181,3900348,88726,441,11022,15027
360,2021-03-07,Россия,4322776,3911924,89094,368,10595,11576
361,2021-03-08,Россия,4333029,3922246,89473,379,10253,10322
362,2021-03-09,Россия,4342474,3932177,89809,336,9445,9931
363,2021-03-10,Россия,4351553,3945527,90275,466,9079,13350


In [4]:
# dt = datetime.datetime.today()
# today = datetime.date(dt.year,dt.month,dt.day)
# ystday = datetime.date(today.year,today.month,today.day-1)
# yst2day = datetime.date(today.year,today.month,today.day-2)
# yst3day = datetime.date(today.year,today.month,today.day-3)

# print(today)
# print(ystday)
# print(yst2day)
# print(yst3day)
myday = list(yandex_russia.tail(1).Date)[0]

print(yandex_russia.iloc[-1,0])
print('Выбран день: ',myday)

2021-03-10
Выбран день:  2021-03-10


In [5]:
X = pd.DataFrame()
regions_dict = {}
yandex_russia['Rt'] = 1
yandex_russia['Rt2'] = 1
yandex_russia['Rate_conf'] = 0
yandex_russia['Rate_ill'] = 0

temp_df = yandex_russia.groupby(['Region','Date']).sum()

# Ниже enumerate  для составления словаря из имен регионов {i:j} , для расчетов можно было использовать только j

for i,j in enumerate(temp_df.index.get_level_values(0).unique()):
    temp_df2 =  temp_df.loc[j]
    temp1 = temp_df2['Day_confirmed'].rolling(4).sum()/(temp_df2["Day_confirmed"].rolling(8).sum()-

                                                              temp_df2["Day_confirmed"].rolling(4).sum())
    temp2 = temp_df2['Confirmed'].rolling(4).sum()/(temp_df2['Confirmed'].rolling(8).sum()-

                                                              temp_df2['Confirmed'].rolling(4).sum())
    temp_df2['Rt'] = round(temp1,3)
    temp_df2['Rt2'] = round(temp2,3)
    temp_df2['Region'] = j
    temp_df2['Remaining_ill']=temp_df2['Confirmed']-temp_df2['Recovered']-temp_df2['Deaths']
    temp_df2['MA7_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=7).mean()
    temp_df2['MA14_dayconfirmed']=temp_df2['Day_confirmed'].rolling(window=14).mean()
    temp_df2['MA7_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=7).mean()
    temp_df2['MA14_remaining_ill']=temp_df2['Remaining_ill'].rolling(window=14).mean()
    
    temp_df2['Change_dayconf'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_dayconf'].iloc[k]=temp_df2['Day_confirmed'].iloc[k]-temp_df2['Day_confirmed'].iloc[k-1]
    temp_df2['Change_remill'] = 0
    for k in range(1,temp_df2.shape[0]):
        temp_df2['Change_remill'].iloc[k]=temp_df2['Remaining_ill'].iloc[k]-temp_df2['Remaining_ill'].iloc[k-1]
    
    

   #Генератор DF  на каждый регион(сидит в цикле):
   # globals()['x' + str(i)] = temp_df2.reset_index()
    
    X = pd.concat([X,temp_df2.reset_index()],axis = 0)
    regions_dict[i]=j
    
print(regions_dict.items())
    
X = X.reset_index(drop=True)
X.tail()


dict_items([(0, 'Адыгея'), (1, 'Алтай'), (2, 'Алтайский край'), (3, 'Амурская обл.'), (4, 'Архангельская обл.'), (5, 'Астраханская обл.'), (6, 'Башкортостан'), (7, 'Белгородская обл.'), (8, 'Брянская обл.'), (9, 'Бурятия'), (10, 'Владимирская обл.'), (11, 'Волгоградская обл.'), (12, 'Вологодская обл.'), (13, 'Воронежская обл.'), (14, 'Дагестан'), (15, 'Еврейская АО'), (16, 'Забайкальский край'), (17, 'Ивановская обл.'), (18, 'Ингушетия'), (19, 'Иркутская обл.'), (20, 'Кабардино-Балкария'), (21, 'Калининградская обл.'), (22, 'Калмыкия'), (23, 'Калужская обл.'), (24, 'Камчатский край'), (25, 'Карачаево-Черкессия'), (26, 'Карелия'), (27, 'Кемеровская обл.'), (28, 'Кировская обл.'), (29, 'Коми'), (30, 'Костромская обл.'), (31, 'Краснодарский край'), (32, 'Красноярский край'), (33, 'Крым'), (34, 'Курганская обл.'), (35, 'Курская обл.'), (36, 'Ленинградская обл.'), (37, 'Липецкая обл.'), (38, 'Магаданская обл.'), (39, 'Марий Эл'), (40, 'Мордовия'), (41, 'Москва'), (42, 'Московская обл.'), (4

,Date,Confirmed,Recovered,Deaths,Day_deaths,Day_confirmed,Day_recovered,Rt,Rt2,Rate_conf,Rate_ill,Region,Remaining_ill,MA7_dayconfirmed,MA14_dayconfirmed,MA7_remaining_ill,MA14_remaining_ill,Change_dayconf,Change_remill
31282,2021-03-06,33567,32115,427,5,120,150,0.896,1.016,0,0,Ярославская обл.,1025,128.428571,140.714286,1130.571429,1218.142857,-3,-35
31283,2021-03-07,33688,32263,431,4,121,148,0.928,1.015,0,0,Ярославская обл.,994,125.714286,137.928571,1096.428571,1193.071429,1,-31
31284,2021-03-08,33805,32400,433,2,117,137,0.932,1.015,0,0,Ярославская обл.,972,123.000000,135.071429,1063.285714,1167.214286,-4,-22
31285,2021-03-09,33917,32534,436,3,112,134,0.934,1.015,0,0,Ярославская обл.,947,120.428571,131.785714,1031.714286,1140.357143,-5,-25
31286,2021-03-10,34031,32664,439,3,114,130,0.941,1.014,0,0,Ярославская обл.,928,119.142857,128.857143,1002.857143,1112.785714,2,-19


In [6]:
mask = X.Region == 'Россия'
temp_df = X[mask][['Date','Confirmed','Remaining_ill']]
temp_df.index = temp_df.Date
temp_df.drop(columns =['Date'], inplace = True)
print(temp_df.tail(2))
# print(temp_df.index.get_level_values(0))
# temp_df.loc[datetime.date(2020,6,15),'Россия'].Confirmed
# for i in temp_df.index.get_level_values(0).unique()[-2:]:
#     print(temp_df.loc[i,'Россия'].Confirmed)
rus_date = list(temp_df.index)
rus_conf = list(temp_df.Confirmed)
rus_ill = list(temp_df.Remaining_ill)
rus_dict_conf = dict(zip(rus_date,rus_conf))
rus_dict_ill = dict(zip(rus_date,rus_ill))

print(len(X))
print(X.shape)
print(X.loc[0,'Date'])
for i in range(len(X)):
    X.loc[i,'Rate_conf']=100*X.loc[i,'Confirmed']/rus_dict_conf.get(X.loc[i,'Date'])
    X.loc[i,'Rate_ill']=100*X.loc[i,'Remaining_ill']/rus_dict_ill.get(X.loc[i,'Date'])


            Confirmed  Remaining_ill
Date                                
2021-03-09    4342474         320488
2021-03-10    4351553         315751
31287
(31287, 19)
2020-03-12


In [7]:
mask = (X.Date>=datetime.date(2020,6,13))&(X.Region.isin(['Москва','Россия']))

temp=X[mask]
temp[['Date','Confirmed','Region','Rate_conf','Rate_ill']].tail(20)

,Date,Confirmed,Region,Rate_conf,Rate_ill
20352,2021-02-19,4139031,Россия,100.0,100.0
20353,2021-02-20,4151984,Россия,100.0,100.0
20354,2021-02-21,4164726,Россия,100.0,100.0
20355,2021-02-22,4177330,Россия,100.0,100.0
20356,2021-02-23,4189153,Россия,100.0,100.0
20357,2021-02-24,4200902,Россия,100.0,100.0
20358,2021-02-25,4212100,Россия,100.0,100.0
20359,2021-02-26,4223186,Россия,100.0,100.0
20360,2021-02-27,4234720,Россия,100.0,100.0
20361,2021-02-28,4246079,Россия,100.0,100.0


In [8]:
# russia_regions = pd.read_csv('regions-info.csv')

# russia_regions[(russia_regions.Region =='Московская область')|(russia_regions.Region =='Москва')|
#               (russia_regions.Region =='Санкт-Петербург')].tail()

In [9]:
X.to_csv('X.csv',index=False)
